In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date, timedelta

In [2]:
## build this dictionary from a table which is read in as a csv - potentially even scrape from the student finance website

historicInterest = {"2012": {"start": date(2012, 9, 1), "end": date(2013, 8, 31), "rate": 0.066}, 
                    "2013": {"start": date(2013, 9, 1), "end": date(2014, 8, 31), "rate": 0.063}, 
                    "2014": {"start": date(2014, 9, 1), "end": date(2015, 8, 31), "rate": 0.055}, 
                    "2015": {"start": date(2015, 9, 1), "end": date(2016, 8, 31), "rate": 0.039}, 
                    "2016": {"start": date(2016, 9, 1), "end": date(2017, 8, 31), "rate": 0.046}, 
                    "2017": {"start": date(2017, 9, 1), "end": date(2018, 8, 31), "rate": 0.0618}, 
                    "2018": {"start": date(2018, 9, 1), "end": date(2019, 8, 31), "rate": 0.063}
                   }


incomeTax = {12500: 0.0, 50000: 0.2, 150000: 0.4, 0.45}
current_wd = os.getcwd()
file_name = "trimmed_data.csv"

def readPayments(file_name, date_header="Payment Date"):
    dataLoc = os.path.join(current_wd, "data", file_name)
    paymentsData = pd.read_csv(dataLoc)
    paymentsData[date_header] = pd.to_datetime(paymentsData[date_header])
    
    return paymentsData

# def findInterestRate():
    

In [4]:
myPayments = pd.read_csv("trimmed_data.csv", sep=",")
myPayments["Payment Date"] = pd.to_datetime(myPayments["Payment Date"])

In [5]:
myPayments

,Payment Date,Instalment,Gross
0,2014-09-25,1,1191.30
1,2014-10-15,1,2250.00
2,2015-01-05,2,1191.30
3,2015-02-04,2,2250.00
4,2015-04-20,3,1227.40
5,2015-05-06,3,4500.00
6,2015-09-24,4,1231.23
7,2015-10-21,4,2250.00
8,2016-01-04,5,1231.23
9,2016-02-03,5,2250.00


In [6]:
def findInterestRate(paymentDate):
    year = paymentDate.year
    sdate = historicInterest[str(year)]["start"]
    edate = historicInterest[str(year)]["end"]
    delta = edate - sdate

    allDates = [sdate + timedelta(days=i) for i in range(delta.days+1)]

    if paymentDate in allDates:
#         print("yes")
        rate = historicInterest[str(year)]["rate"]
#         print(rate)
    else:
        rate = historicInterest[str(year-1)]["rate"]
#         print("next year", rate)
    return rate

In [7]:
def populateInterestRate(myPayments):
    myPayments["interestRate"] = 0
    myPayments["totalPayment"] = 0
    for i, row in myPayments.iterrows():
    #     print(row)
        rate = findInterestRate(myPayments.at[i, "Payment Date"].date())
#         print(rate)
        myPayments.loc[i,"interestRate"] = rate
#         print(rate+1)
        myPayments.loc[i, "totalPayment"] = myPayments.loc[i, "Gross"]*(rate+1)
    return myPayments

In [8]:
myPayments = populateInterestRate(myPayments)

graduationAmount = myPayments.sum()["totalPayment"]*1.061
graduationplusOne = graduationAmount*1.063

In [9]:
myPayments

,Payment Date,Instalment,Gross,interestRate,totalPayment
0,2014-09-25,1,1191.30,0.055,1256.82150
1,2014-10-15,1,2250.00,0.055,2373.75000
2,2015-01-05,2,1191.30,0.055,1256.82150
3,2015-02-04,2,2250.00,0.055,2373.75000
4,2015-04-20,3,1227.40,0.055,1294.90700
5,2015-05-06,3,4500.00,0.055,4747.50000
6,2015-09-24,4,1231.23,0.039,1279.24797
7,2015-10-21,4,2250.00,0.039,2337.75000
8,2016-01-04,5,1231.23,0.039,1279.24797
9,2016-02-03,5,2250.00,0.039,2337.75000


In [11]:
def salaryIncrease(salary):
    increaseStep = np.random.normal(7000, 2000)

    return salary+increaseStep

In [ ]:
def calcNetSalary(grossSalaryPA, student_finance = True):
    '''calculates take home salary per year based on 
    UK tax bands'''
    if !student_finance:
        if grossSalaryPA =< 12000:
            netSalaryPA = 12000
        elif grossSalaryPA >= 12000 and grossSalaryPA:
            taxable
            netSalaryPA = (0.8*gro)
    else:
        

In [43]:
salaryIncrease(salary=30000)

35470.15291461571

# To-do
* Function to calculate net pay after income tax based on different bands and tax-free allowance
* Function to calculate student finance owed - need this as separate so can calculate cumulative total over simulated years
* 